# Intro2ML 2023 - Classification
## Nicola Dall'Asen
### nicola.dallasen@unitn.it
In this notebook we will learn by examples how to perform classification using different algorithms:
- KNN
- Decision trees
- Random forests
- SVM

## Classification of handwritten digits
In this document we address a classification task. The goal is to find a function $f\in\mathcal Y^\mathcal X$ that given an input $x\in\mathcal X$ provides a class label $f(x)\in\mathcal Y$ in a way that to best fit some pre-defined (unknown) data distribution on $\mathcal X\times\mathcal Y$. More specifically, we address the problem of classifying handwritten digits. Accordingly, the input space $\mathcal X$ consists of gray-scale images of fixed resolution $h\times w$ representing handwritten digits from $0$ to $9$, and the label space is given by $\mathcal Y=\{0,\ldots,9\}$. Below we see some examples:
![Examples from MNIST](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

To get a representative sample of the distribution of handwritten digits, we consider a well-known dataset in the field called [MNIST](https://en.wikipedia.org/wiki/MNIST_database), which contains $70$k gray-scale images of handwritten digits from $0$ to $9$, each image being $28\times 28$ (so, $w=h=28$). 

Our goal in this document is to play with different machine learning algorithms, rather than pursuing the best absolute performance on this dataset. Accordingly, we will consider a subset of $20$k images that we split $10$k images for training, $5$k for validation and $5$k for test.



## MNIST

We start by loading and inspecting the MNIST dataset. To this end, we make use of scikit-learn to fetch MNIST's data. Note that the following instructions will take several seconds for the execution.

In [ ]:
# Import the function to fetch MNIST from scikit-learn
from sklearn.datasets import fetch_openml

# Fetching MNIST data by returning digit images and targets in two tensors
# We call the tensor with input data X and the relative targets y
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

We next inspect the tensors we have and convert them from NumPy to PyTorch tensors, since we learnt to use them.

In [ ]:
# Here we print the shapes of the two returned tensors
print(f"Shape of input data: {X.shape}")
print(f"Shape of targets: {y.shape}")
# Here we inspect the ranges of the elements of X and y
print(f"Minimum and maximum values in X. Min={X.min()} Max={X.max()}")

# Here we inspect the data types of the NumPy tensors
print(f"Data type of X: {X.dtype}")
print(f"Data type of y: {y.dtype}")

From what we printed above we deduct the following information. The size of the input tensor $X$ is $70$k $\times 28^2$, i.e. each row of $X$ represents the $28\times 28$ digit image, linearized. The range of gray-scale values is $[0,255]$. The entries of $X$ are `float64`. As for $y$ we have one entry for each image, holding the relative class label. However the data type of the elements of $y$ is `object`. To actually figure out the real datatype, we inspect the content and type of the first element.

In [ ]:
# Since the data type of y is not numeric but generic, we inspect the
# single elements
print(f"Content of y[0]: {y[0]}, type of y[0]: {type(y[0])}")

We will now use matplotlib to visualize some digits with the corresponding label.

In [ ]:
# Import libraries for plotting
import matplotlib.pyplot as plt

# Here we plot the first 10 images in the dataset on two rows with their labels
fig, ax = plt.subplots(2, 5, figsize=(10, 5))
for i in range(10):
    ax[i // 5, i % 5].imshow(X[i].reshape(28, 28), cmap="gray")
    ax[i // 5, i % 5].axis("off")
    ax[i // 5, i % 5].set_title(y[i])
plt.show()

Next, we have to reduce the number of samples from $70$k to $20$k since we will work with a reduced set.

In [ ]:
import numpy as np

# We keep track of the original data
X_full = X
y_full = y

# We select a subset of random 20k training samples
n_samples = 20000
idx = np.random.choice(X.shape[0], n_samples, replace=False)
X = X[idx]
y = y[idx]

Then, we have to split the data into a training set, validation set and test set of $10$k, $5$k and $5$k, respectively

In [ ]:
# Compute the size of the three sets: training, validation ,test
split_sizes = [10000, 5000, 5000]

# We split the tensor according to the specified sizes using torch.split
X_train, X_val, X_test = np.split(X, np.cumsum(split_sizes)[:-1])
y_train, y_val, y_test = np.split(y, np.cumsum(split_sizes)[:-1])

# Here we print the shapes of the three sets
print(f"Shape of training set: {X_train.shape}")
print(f"Shape of validation set: {X_val.shape}")
print(f"Shape of test set: {X_test.shape}")

To recap, the training set images are stored as rows of the 2D vector $X_\text{tr}$ and the relative class labels are stored in the 1D vector $y_\text{tr}$. Similarly we have the validation set in $X_\text{val}$ and $y_\text{val}$, and the test set in $X_\text{ts}$ and $y_\text{ts}$.

## Evaluation metrics
We consider three evaluation metrics:
* Global Accuracy (Acc): $\frac{\text{correctly classified images}}{\text{of images}}$
* Class Averaged Accuracy (mAcc)
* Class averaged Intersection-over-Union (mIoU)

We write a function to compute the three error metrics.

In [ ]:
import sklearn


def evaluate(yt, yp, num_classes=10):
    # Computes the three metrics Acc, mAcc and mIoU
    # yt: 1D tensor of size n with the target class labels
    # yp: 1D tensor of size n with the predicted class labels
    # num_classes: total number of classes

    # Compute the confusion matrix C, where C_ij represents the number of images
    # of class i classified with class j
    # C = np.zeros((num_classes, num_classes))
    # for i in range(yt.shape[0]):
    #   C[yt[i], yp[i]] += 1

    # Alternatively one can use sklearn.metrics.confusion_matrix(yt,yp)
    C = sklearn.metrics.confusion_matrix(yt, yp)

    return {
        # The diagonal of C holds the images that have been correctly classified, so
        # Acc can be computed as the sum of the diagonal divided by the number of images
        "Acc": C.diagonal().sum().item() / C.sum().item(),
        # For mAcc we need to divide the diagonal of C by the sum of each row of C
        # because the latter represents the number of images per class. This yields
        # a tensor with the per-class accuracies. Finally, we average the result to
        # get the mean per-class accuracy.
        "mAcc": (C.diagonal() / C.sum(-1)).mean().item(),
        # For the IoU computation we divide the diagonal of C by the sum of the columns
        # of C plus the sum of the rows of C minus the diagonal of C. This yeilds per-class
        # IoUs, which are finally averaged.
        "mIoU": (C.diagonal() / (C.sum(0) + C.sum(1) - C.diagonal())).mean().item(),
    }

## Machine learning classification models
We will train the following machine learning models using scikit-learn, which have been addressed during the course:

* Nearest Neghbour Classifier (seen last time)
* Decision Tree Classifier
* Random Forest
* Support Vector Machines

For each model we will make an analysis of its performance.

### Nearest Neighbor (NN) Classifier

A Nearest Neighbor classifier is a non-parametric model that has no learnable parameters, but uses the entire training set every time it requires to perform a prediction. Indeed, given an input sample to predict, the NN classifier first retrieves a number of training samples that are close to the one we intend to predict, according to a predefined distance metric, and determines the output class from a possibly-weighted voting procedure. In order to perform inference in an efficient way, the NN classifier adopts typically tree-based datastructures to speedup the search for neighbors. 

In scikit-learn we can create a nearest neighbor classifier using the class [`neighbors.KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier) or [`neighbors.RadiusNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.RadiusNeighborsClassifier.html#sklearn.neighbors.RadiusNeighborsClassifier) depending on the type of NN algorithm we want to use. The former one searches for a fixed number of $K$ closest neighbors, while the latter searches for neighbors within a maximum radius.

In [ ]:
# Import required libraries
from sklearn import neighbors

# Visualize help of KNN
neighbors.KNeighborsClassifier?

The constructor of `neighbors.KNeighborsClassifier` admits different arguments. The most relevant ones are `n_neighbors`, which is the desired number of neighbors $K$, `weights` (uniform or distance or user-defined) which specifies how the importance of a neighbor should be weighted, `metric` the distance metric to use.

We start by creating a KNN classifier with default parametrization.

In [ ]:
# We initialize the nearest neighbor classifier.
# Since inference can be very slow, we enable the use of multiple processes with n_jobs=-1
knn_model = neighbors.KNeighborsClassifier(n_jobs=-1)

Next we run [`fit`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier.fit) on the training data in order to "train" the classifier. This will trigger the creation of the necessary datastructures that allows to quickly identify nearest neighbors during inference.

In [ ]:
# We will track the training/inference times of each method
import time

exec_times = {}
scores = {}
# We train a KNN model and track time
# It takes some seconds
start = time.time()
knn_model.fit(X_train, y_train)
end = time.time()

exec_times["train"] = {"kNN": end - start}

print(f"Training time: {exec_times['train']['kNN']:.2f} seconds")

We take now a random validation digit, visualize it and predict it with [`predict`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier.predict).

In [ ]:
query_idx = np.random.randint(0, X_val.shape[0])
sample = X_val[query_idx]

# Compute the prediction for the first validation image
prediction_knn = knn_model.predict(sample.reshape(1, -1))[0]

sample = sample.reshape(28, 28)
# Plot the digit and in the title we report the ground-truth label and the predicted one
plt.imshow(sample, cmap="gray")
plt.title(f"Ground-truth: {y_val[query_idx]}, prediction: {prediction_knn}")
plt.axis("off");

Next we inspect what were the nearest neighbors for this validation image. We can use the method [`kneighbors`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier.kneighbors). The latter function returns the indices of the nearest neghbors and their distance to the query points.

In [ ]:
def plot_neighbors(knn_model, query):
    # Compute the nearest neighbors
    query = query.reshape(1, -1)
    neighbors_dist, neighbors_idx = knn_model.kneighbors(query)

    # We Visualize them
    for i, (d, idx) in enumerate(zip(neighbors_dist[0], neighbors_idx[0])):
        ax = plt.subplot(1, len(neighbors_idx[0]), i + 1)
        ax.axis("off")
        plt.imshow(X_train[idx].reshape(28, 28), cmap="gray")
        plt.title(f"{y_train[idx]}")

    print(f"Distance to neighbors: {neighbors_dist[0]}")


plot_neighbors(knn_model, X_val[query_idx])

Due to the slowness of the inference procedure, we skip the use of the validation set for model selection (e.g. find an optimal value of the number of neighbors).

We will now predict the entire training and validation set and report performances.

In [ ]:
# Compute prediction for the training and validation set and track time
knn_prediction_train = knn_model.predict(X_train)

start = time.time()
knn_prediction_val = knn_model.predict(X_val)
end = time.time()

exec_times["val"] = {"kNN": end - start}

# Print the validation performance
print(f"Performance on training: {evaluate(y_train, knn_prediction_train)}")
print(f"Performance on validation: {evaluate(y_val, knn_prediction_val)}")
print(f"Evaluation time: {exec_times['val']['kNN']:.2f} seconds")

In [ ]:
def visualize_failure_cases(X, yt, yp, num_failures_to_show=6):
    # Visualizes failure cases and return the indices of the visualized images
    # X: input data as a 2D tensor n x 28**2, each row being a linearized 28x28 image
    # yt: 1D tensor with n ground-truth labels
    # yp: 1D tensor with n predictions

    # returns indices of images that have been visualized

    # We determine which images are misclassified and store this in a boolean 1D tensor
    failures = yt != yp
    # We look for the indices of misclassified images and retain only the first num_failures_to_show ones.
    failures_to_show = np.where(failures)[0][:num_failures_to_show]

    plt.figure(figsize=(12, 12))

    # We plot the images where we are failing. i.e. the one  with indec in failures_to_show
    # Each plot shows prediction vs target as title
    for i, failure_idx in enumerate(failures_to_show):
        ax = plt.subplot(1, num_failures_to_show, i + 1)
        ax.axis("off")
        plt.imshow(X[failure_idx].reshape(28, 28), cmap="gray")
        plt.title(f"GT: {yt[failure_idx]}, P: {yp[failure_idx]}")
    return failures_to_show

We will next inspect some failure cases.

In [ ]:
# This visualizes some failure cases
failure_cases = visualize_failure_cases(X_val, y_val, knn_prediction_val)

Next, we visualize the nearest neighbors of the first failure case in the list.

In [ ]:
# Visualize nearest neighbors of the failure case
plot_neighbors(knn_model, X_val[failure_cases[0:1]])

Finally we compute the scores for the test set.

In [ ]:
# Compute prediction for the test set
# It takes roughly 1 minute.
knn_prediction_test = knn_model.predict(X_test)

scores["kNN"] = evaluate(y_test, knn_prediction_test)

# Print the test performance
print(f"Performance on test: {scores['kNN']}")

### Decision Tree (DT) Classifier
A decision tree classifier can be defined in scikit-learn using the class [tree.DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html?highlight=decision%20tree#sklearn.tree.DecisionTreeClassifier).

We start by importing the library and inspect the arguments of the constructor.

In [ ]:
# Import required library
from sklearn import tree

#This opens the documentation about the class
tree.DecisionTreeClassifier?

The instructions above import the required libraries and show the documentation about the class. We can see that when we create a `DecisionTreeClassification` object we can specify with `criterion` the quality measure (gini and entropy are available), whether we want to find the best (axis-aligned) split or a random one (see `splitter`), if we want to put conditions to stop the growth (`max_depth`, `min_samples_split`, `min_samples_leaf`, `max_leaf_nodes`, `min_impurity_decrease`, `min_impurity_split`), and other arguments.

We start creating a decision tree with the default parametrizations.

In [ ]:
dtree = tree.DecisionTreeClassifier()

Similarly to what we have seen in the polynomial regression experiment, also this scikit-learn model exposes a function [`fit`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html?highlight=decision%20tree#sklearn.tree.DecisionTreeClassifier.fit) that can be used to train the model. 

In [ ]:
# We will track the training/inference times of each method
import time

# Trains a decision tree on the training set
# The execution takes few seconds
start = time.time()
dtree.fit(X_train, y_train)
end = time.time()

exec_times["train"]["DT"] = end - start

print(f"Training time: {exec_times['train']['DT']:.2f} seconds")

We can now gather some information about the trained model, in particular depth and number of leaves.

In [ ]:
# Depth
print(f"Depth: {dtree.get_depth()}")

# Number of leaves
print(f"Number of leaves: {dtree.get_n_leaves()}")

We can further inspect the trained tree which is stored in `dtree.tree_`. We refer to the documentation about [tree structure](https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html#sphx-glr-auto-examples-tree-plot-unveil-tree-structure-py) for more details. 

The tree is stored in terms of several arrays, being fields of `dtree.tree_`:
* `children_left`: array where the $i$th entry represents the index of the left child of node $i$
* `children_right`: same but for the right child
* `node_count`: number of nodes (leaves included) in the tree
* `feature`: array where the $i$th entry represents the index of the feature used by the split function in node $i$ (a negative value for leaves)
* `threshold`: array where the $i$th entry represents the threshold used by the split function in node $i$ 
* `value`: array where the $i$th entry holds the class probability distribution of training samples reaching that node 

As an example we will inspect the rule that has been applied to split the data at the root node. 

In [ ]:
print(f"Feature used to split @ root: {dtree.tree_.feature[0]}")
print(f"Threshold used to split @ root: {dtree.tree_.threshold[0]}")

We can visualize the position of the selected feature by plotting a black point there.

In [ ]:
# Make an image with all ones
I = np.ones((28, 28))
# Set to zero the feature selected for the root
# To this end we need to view the 2D tensor as a 1D tensor and set to 0 the
# entry corresponding to the selected feature
I.reshape(-1)[dtree.tree_.feature[0]] = 0

# Plot the image
plt.imshow(I, cmap="gray")
plt.title("Position of the feature used to split data in the root node");

Next, we can plot the distribution of the selected features across all nodes, to have an idea of where the most used features are.

In [ ]:
# The following two lines of code generate in feat_distrib a 1D tensor of
# size 28**2, where each entry, say j, counts the number of occurences of the
# jth feature
feat_distrib = np.zeros(28**2)
for i in range(dtree.tree_.feature.shape[0]):
    if dtree.tree_.feature[i] >= 0:
        feat_distrib[dtree.tree_.feature[i]] += 1

# We visualize the distribution
plt.imshow(feat_distrib.reshape(28, 28), cmap="gray")

The saliency of the features correlates with where the digits are typically placed within the image. This way of computing the importance of the features does not take into account how many samples of the training set were split by a given feature, e.g. the split decision in the root is way more important than a split decision close to a leaf. A better estimate of the importance requires weighting each feature by the number of samples thatit splits. This can be obtained from scikit-learn using the field `dtree.feature_importances_`, which in our case is a $28**2$ long 1D tensor of weights.

We visualize the properly-weighted importance of the features below.

In [ ]:
# We visualize the feature importance after properly reshaping it in a 28x28 2D tensor
plt.imshow(dtree.feature_importances_.reshape(28, 28), cmap="gray")
plt.title("Normalized feature importance");

Next we inspect the class distribution of the training set after the split in the root node.

In [ ]:
# Creates a histogram of class labels of images routed left at the root

# We determine the indices of the images routed left at the root
left_idx = np.where(X_train[:, dtree.tree_.feature[0]] < dtree.tree_.threshold[0])[0]

# We determine the class labels of the images routed left at the root
left_labels = y_train[left_idx]
right_labels = y_train[~left_idx]

# count each class label, knowing that the labels are strings
left_labels_count = np.unique(left_labels, return_counts=True)
right_labels_count = np.unique(right_labels, return_counts=True)

# We plot the histogram as difference between the two distributions
dist = left_labels_count[1] - right_labels_count[1]
plt.bar(left_labels_count[0], dist)
plt.title("Difference in class distribution between left and right branches")

We can additionally try to visualize the average image of digits being routed to the left and to the right.

In [ ]:
# Compute the average digit from images routed to the left, X_train is numpy array
mean_left = np.mean(X_train[left_idx], axis=0)
mean_right = np.mean(X_train[~left_idx], axis=0)

# Plot the average digits in a subplot
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(mean_left.reshape(28, 28), cmap="gray")
plt.title("Average digit routed left")
plt.subplot(1, 2, 2)
plt.imshow(mean_right.reshape(28, 28), cmap="gray")
plt.title("Average digit routed right")

We compute now how the trained model performs on the training data and validation data. To this end, we use the method [`predict`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.predict) to classify the training and validation data and use the evaluation function we have defined to compute the performance.

In [ ]:
# Compute predictions on training data
ytr_p = dtree.predict(X_train)

# Compute predictions on validation data and track execution time
start = time.time()
yval_p = dtree.predict(X_val)
end = time.time()
exec_times["val"]["DT"] = end - start

# Print results
print(f"Performance on training: {evaluate(y_train, ytr_p)}")
print(f"Performance on validation: {evaluate(y_val, yval_p)}")
print(f"Evaluation time: {exec_times['val']['DT']:.2f} seconds")

We know that decision trees are prone to overfitting unless we control the complexity of the model in some way. Indeed, the performance of training is perfect, but we loose accuracy on the validation set.

We will now inspect some failure cases to understand whether those are due to ambiguous digits or bad generalization.

We will now try to train a decision tree by restricting the minimum number of samples that we have in a leaf. We will try different values of this hyperparameter. We will also consider the possibility of using a different quality measure, i.e. entropy (gini is the default).

In [ ]:
# We train 4 different models, i.e. each combination of min_samples_leaf in {1,5} and criterion in {"gini,"entropy"}
# This will take few seconds.
dtrees = [
    tree.DecisionTreeClassifier(min_samples_leaf=ms, criterion=criterion).fit(
        X_train, y_train
    )
    for ms in [1, 5]
    for criterion in ["gini", "entropy"]
]

 We measure then the performance of each model on the validation set in order to select the best, along with the performance on the training set.

In [ ]:
# We compute the predictions of each tree on the validation set in a list
dtree_predictions_val = [dt.predict(X_val) for dt in dtrees]

# We compute the performance of each model on the validation set in a list
dtree_performance_val = [evaluate(y_val, yp) for yp in dtree_predictions_val]

plt.figure(figsize=(12, 12))
# We plot the performance of each model on each metric
for i, metric in enumerate(["Acc", "mAcc", "mIoU"]):
    plt.subplot(3, 1, i + 1)
    curve_val = [res[metric] for res in dtree_performance_val]
    plt.plot(curve_val, "*", markersize=10)
    plt.grid(axis="y")
    plt.ylabel(metric)
    # This removes the labels from the x-axis
    plt.xticks([])

# We give a name to each model we have:
# G1: criterion=gini, num_samples_leaf=1
# E1: criterion=entropy, num_samples_leaf=1
# G5: criterion=gini, num_samples_leaf=5
# E5: criterion=entropy, num_samples_leaf=5
dt_model_names = ["G1", "E1", "G5", "E5"]

# We set the x-axis labels only for the last plot
plt.xlabel("models")
plt.xticks(range(4), dt_model_names)

We can now identify the model yielding the best validation performance, although there are little differences across the models. We observe that the use of the entropy criterion improves over Gini when we have an unconstrained tree. We can now check whether the best model improved over the previously selected failure cases. 


In [ ]:
# We select the best model based on the mAcc metric
best_dt_model_idx = np.argmax([res["mAcc"] for res in dtree_performance_val])

failures_to_show = visualize_failure_cases(X_val, y_val, yval_p)
# Each plot shows prediction vs target
for i, failure_idx in enumerate(failures_to_show):
    ax = plt.subplot(1, len(failures_to_show), i + 1)
    ax.axis("off")
    plt.imshow(X_val[failure_idx].reshape(28, 28), cmap="gray")
    plt.title(
        f"P: {dtree_predictions_val[best_dt_model_idx][failure_idx]} vs GT: {y_val[failure_idx]}"
    )

Indeed, we provide now some correct answers for some previous failure cases, but new errors could have popped up for other images. Nonetheless, from the plot above we know that overall there was an improvement on the validation set. However, this does not guarantee that the selected model achieves a better generalization error, because the selection inherited some bits of information about the validation set, thus biasing the model towards it.
To get a better (unbiased) estimate of the generalization performance we rely on the test set.

In [ ]:
# We compute the predictions on the test set for the best model
test_predictions_dt = dtrees[best_dt_model_idx].predict(X_test)

# We keep the scores of the selected model in scores
# This will later hold also scores of different types of models
scores["DT"] = evaluate(y_test, test_predictions_dt)

# We print the performance on the test set for the selected decision tree.
print(f"Performance on test set: {scores['DT']}")

This is the score that we will use to compare against other models.

### Random Forest (RF)

A single decision tree tends to overfit the training data, unless we reduce the tree complexity, but also in that case, the algorithm used to grow the tree does not guarantee to find a model with minimal complexity. We have seen that for this specific example, the heuristic of stopping the tree growth based on the minimum number of samples in the leaf was of little/no effect. 

A better approach to improve on the generalization performance is by means of ensembles of possibly uncorrelated low-bias models. Decision trees are great candidates models to form an ensemble and in order to render them less correlated we can train them by randomly sampling features and by perturbing the training set for each tree via e.g. bagging. 

Random Forest is a popular machine learning model that consists in an ensemble of decision trees. We can create such a model in scikit-learn via [`ensemble.RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn-ensemble-randomforestclassifier). 



In [ ]:
# Import required libraries
from sklearn import ensemble

#Visualize help
ensemble.RandomForestClassifier?

This class shares the same arguments for the constructor that we have seen for decision trees, but has also additional ones such as `n_estimators`, which holds the number of trees we want to have in the ensemble, and `boostrap`, which enables boostrapping of the training set when growing each tree (enabled by default).

We have seen that the best criterion seems to be entropy over gini. Moreover, ensembles are more effective if the single models have low bias, so we do not enforce stopping criteria that reduce the complexity of single trees. We set the number of trees in the ensemble to $20$ in order to keep the training time short.

In [ ]:
# We create a random forest model with 10 trees and entropy criterion
rf_model = ensemble.RandomForestClassifier(n_estimators=20, criterion="entropy")

Next, we train the forest on the training data using the usual [`fit`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.fit) function.

In [ ]:
# We train the random forest model on the training set and track training time
start = time.time()
rf_model.fit(X_train, y_train)
end = time.time()

exec_times["train"]["RF"] = end - start

print(f"Training time: {exec_times['train']['RF']:.2f} seconds")

Each trained tree can be accessed in the list `rf_model.estimators_`.

We compute next the performance on training and validation using the [`predict`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict) method to classify, which internally computes the prediction of each tree in the ensemble and performs a probability-weighted voting.

In [ ]:
# Compute prediction for the training and validation sets and track time of validation
rf_prediction_train = rf_model.predict(X_train)

start = time.time()
rf_prediction_val = rf_model.predict(X_val)
end = time.time()

exec_times["val"]["RF"] = end - start

# Print the training and validation performance
print(f"Performance on training: {evaluate(y_train, rf_prediction_train)}")
print(f"Performance on validation: {evaluate(y_val, rf_prediction_val)}")
print(f"Evaluation time: {exec_times['val']['RF']:.2f} seconds")

As we can see the performance on validation is considerably higher than what we got with a single decision tree. Also, the performance on the training set is not perfect, but almost, due to the presence of bagging, which might leave some data out from the training set of each tree.

Next, we visualize some failure cases.

In [ ]:
# This visualizes some failure cases
visualize_failure_cases(X_val, y_val, rf_prediction_val)

We will now change the number of trees in the ensemble in the range $\{3,\ldots,20\}$ and report the validation performance.

In [ ]:
# Make a temporary copy of all trees in the forest
all_trees = rf_model.estimators_

# Init the forest model with only the first 2 trees
rf_model.estimators_ = [all_trees[i] for i in range(2)]

# We will now iteratively add one tree at time and compute the score
# in order to report the performance as we add more trees to the
# ensemble

curve_val = {"Acc": [], "mAcc": [], "mIoU": []}

# For all remaining trees
for i in range(2, len(all_trees)):
    # We add the next tree
    rf_model.estimators_.append(all_trees[i])

    # Compute the prediction on validation of the updated ensemble
    rf_prediction_val = rf_model.predict(X_val)

    # Evaluate the result and update the Acc, mAcc and mIoU curves
    res = evaluate(y_val, rf_prediction_val)
    for k in res:
        curve_val[k].append(res[k])

plt.figure(figsize=(12, 12))
# Finally we plot the curves for each metric
for i, metric in enumerate(curve_val.keys()):
    plt.subplot(3, 1, i + 1)
    plt.plot(range(3, len(all_trees) + 1), curve_val[metric], linewidth=2)
    plt.ylabel(metric)
    # We keep the x-axis only for the last plot
    plt.xticks([])
    plt.grid(axis="y")
plt.xlabel("number of trees")
plt.xticks(range(3, len(all_trees) + 1));

As the number of trees in the ensemble increases, the performance on validation increases. 

We finally report the score on the test set.

In [ ]:
# Compute predictions for the test set
rf_prediction_test = rf_model.predict(X_test)

# Print the
scores["RF"] = evaluate(y_test, rf_prediction_test)
print(f"Performance on test: {scores['RF']}")

### Support Vector Machine (SVM)

Finally, we evaluate the performance of Support Vector Machines, namely robust linear classifiers that can become non-linear by employing suitable kernel functions. SVMs are binary by nature, but can deal with multiple classes by casting the multi-class classification problem into a sequence of binary classification problems (one-vs-all, all-vs-all, and others).

In scikit-learn there are different classes that can be used to train SVMs for classification among which we find [`svm.SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC), [`svm.NuSVM`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.NuSVC.html#sklearn.svm.NuSVC) and [`svm.LinearSVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC). We will use the first one, which is the SVM formulation presented during the course and can deal with kernels, as opposed to `LinearSVC`, which is faster but implements only linear SVMs. The second one is a different SVM formulation which has not been presented during the course, so it will be skipped.

In [ ]:
# Import required libraries
from sklearn import svm

# Visualize help
svm.SVC?

The relevant arguments for the constructor are `C`, which balances the importance of the regularizer (inversely proportional), `kernel` (linear, poly, rbf, sigmoid or user-specified) which allows to specify the desired kernel function. Based on the kernel selection there are a number of kernel-specify parameters. Multi-class predictions are addressed with a all-vs-all strategy.

We start by initializing a C-SVM with default parametrization.

In [ ]:
# Intialize a C-SVM model
svm_model = svm.SVC()

Next, we train the SVM model on the training set, using the usual [`fit`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.fit) function.

In [ ]:
# Train the C-SVM on the training set and track time
# It takes several seconds
start = time.time()
svm_model.fit(X_train, y_train)
end = time.time()

exec_times["train"]["SVM"] = end - start

print(f"Training time: {exec_times['train']['SVM']:.2f} seconds")

We can inspect the trained model using the following fields:
* `support_`: array of indices of support vectors
* `support_vectors_`: array of training samples representing the support vectors
* `n_support_`: array with the number of support vectors for each class
* `coeff_`: weights of the SVM solution (only for linear kernel)
* `dual_coeff_`: dual variables of the SVM solution
* `intercept_`: constant in the decision function

We will print the number of support vectors.

In [ ]:
print(f"Number of support vectors per class: {svm_model.n_support_}")

Next, we compute the performance on both the training and validation sets using the [`predict`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.predict) function.

In [ ]:
# Compute prediction for the training and validation set and track evaluation time
# It takes a bit more than 1 minute.
svm_prediction_train = svm_model.predict(X_train)

start = time.time()
svm_prediction_val = svm_model.predict(X_val)
end = time.time()

exec_times["val"]["SVM"] = end - start

# Print the training and validation performance
print(f"Performance on training: {evaluate(y_train, svm_prediction_train)}")
print(f"Performance on validation: {evaluate(y_val, svm_prediction_val)}")
print(f"Evaluation time: {exec_times['val']['SVM']:.2f} seconds")

We visualize some failure cases.

In [ ]:
failures_to_show = visualize_failure_cases(X_val, y_val, svm_prediction_val);

Now, we train several SVM models using different kernels and values of $C$.

In [ ]:
# We initialize and train models with all combinations of kernel in {rbf, poly, linear}
# and C in {0.1,1,10}. For each kernel we keep the standard parametrization
# It takes almost 6 minutes
svm_models = [
    svm.SVC(kernel=kernel, C=C).fit(X_train, y_train)
    for kernel in ["rbf", "linear", "poly"]
    for C in [0.1, 1, 10]
]

We compute the performance on the validation set.

In [ ]:
# Compute predictions on validation for each model
# It takes 3/4 minutes

svms_predictions_val = [svm.predict(X_val) for svm in svm_models]

# Evaluate the performance of each model
svms_performance_val = [evaluate(y_val, yp) for yp in svms_predictions_val]

plt.figure(figsize=(12, 12))
# Produce the plot about the performance of each model on each metric
for i, metric in enumerate(["Acc", "mAcc", "mIoU"]):
    plt.subplot(3, 1, i + 1)
    curve_val = [res[metric] for res in svms_performance_val]
    plt.plot(curve_val, "*", markersize=10)
    plt.ylabel(metric)
    plt.grid(axis="y")
    plt.xticks([])

svm_model_names = [
    "R.1",
    "R1",
    "R10",
    "L.1",
    "L1",
    "L10",
    "P.1",
    "P1",
    "P10",
]
plt.xticks(range(9), svm_model_names)
plt.xlabel("SVM models");

We select the best model and check the performance on the selected failure cases of the previous model.

In [ ]:
# We select the best model based on the mAcc metric
best_svm_model_idx = np.argmax([res["mAcc"] for res in svms_performance_val])

# We plot the images where we failing previously (stored in failures_to_show)
# Each plot shows prediction vs target
plt.figure(figsize=(15, 15))
for i, failure_idx in enumerate(failures_to_show):
    ax = plt.subplot(1, len(failures_to_show), i + 1)
    ax.axis("off")
    plt.imshow(X_val[failure_idx].reshape(28, 28), cmap="gray")
    plt.title(
        f"P: {svms_predictions_val[best_svm_model_idx][failure_idx]} vs GT: {y_val[failure_idx]}"
    )

Finally we compute the performance of the best model on the test set.

In [ ]:
# We compute the predictions on the test set for the best model
# It takes several seconds
test_predictions_svm = svm_models[best_svm_model_idx].predict(X_test)

scores["SVM"] = evaluate(y_test, test_predictions_svm)

# We print the performance on the test set for the selected svm model.
print(f"Performance on test: {scores['SVM']}")

# Comparisons
We conclude with a number of scatter plots comparing the performance of the different best models for each performance metric against the time that was required for training and inference.

In [ ]:
# We retrieve the model names from the keys of the scores dictionary
models = list(scores.keys())

plot_idx = 1

plt.figure(figsize=(15, 15))

# We produce a scatter plot for each combination of metric on the y-axis and
# training/validation time on the x-axis
for metric in ["Acc", "mAcc", "mIoU"]:
    for dataset in ["train", "val"]:
        # We arrange the plots into a 3x2 grid
        plt.subplot(3, 2, plot_idx)
        s = np.array([scores[model][metric] for model in models])
        t = np.array([exec_times[dataset][model] for model in models])

        # s = torch.tensor([scores[model][metric] for model in models])
        # t = torch.tensor([exec_times[dataset][model] for model in models])

        plt.plot(t.reshape(1, -1), s.reshape(1, -1), "*")

        # This parts adds a label to each point in the scatter point, which in turn
        # corresponds to a particular model
        for i in range(len(models)):
            plt.annotate(models[i], (t[i], s[i]))

        # This adds some margin between axis and the points in the plot
        plt.margins(0.3)

        # The following if-the-else removes the y-axis from the second column of
        # plots
        if dataset == "train":
            plt.ylabel(metric)
        else:
            plt.yticks([])

        # The following instruction enables the xaxis only for the last row of plots
        if metric == "mIoU":
            plt.xlabel("{} time [s]".format(dataset))
        else:
            plt.xticks([])

        plot_idx += 1

In terms of accuracy, SVM wins the competition, although we did not conduct an extensive search of models for each type of classifier, however, it requires much longer training time compared to all other methods and longer inference time compared to RF and DT. kNNs achieve also good performance but at the cost of extremely slow inference. DTs can be trained fast and also inference is fast. However, a single tree does not generalize well. RFs, which provide an esemble of trees, provide a good trade-off of accuracy and training/inference speed. Note also that we limited ourself to $20$ trees, but larger ensembles could have been used.

# Exercises

1. Try to improve the performance of the models by tuning the hyper-parameters. You can use the validation set for this purpose.
2. Try other datasets, like:
   1. (reasonably ok) CIFAR-10, which is a dataset of $32\times32$ color images with $10$ classes. The dataset can be downloaded from [here](https://www.cs.toronto.edu/~kriz/cifar.html).
   2. (easy to load, hard to train) CIFAR-100, which is a dataset of $32\times32$ color images with $100$ classes. The dataset can be downloaded from [here](https://www.cs.toronto.edu/~kriz/cifar.html).
   3. (hard to load, easy to train) IMDB, which is a dataset of movie reviews with $2$ classes (positive/negative). The dataset can be downloaded from [here](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews). We are dealing with text data, so you will need to use a different feature extraction method, such as bag-of-words or word embeddings. A good starting point is `scikit-learn`'s [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) classes and `nlkt`'s [`word_tokenize`](https://www.nltk.org/api/nltk.tokenize.html#nltk.tokenize.word_tokenize) function.

No exercise is mandatory, but you are encouraged to try them out. You will learn a lot by doing so!